In [ ]:
import pickle
import pandas as pd
with open('alexnet.fea', 'rb') as f:
    alex_features = pickle.load(f)

train_metadata_df = pd.read_csv('dicom_metadata_train_merged.csv') # Took it from kaggle notebooks


In [ ]:
from torch.utils.data import Dataset
#from torchvision.io import read_image toooo old fot that stuff



class featuresDataset(Dataset):
    def __init__(self, merged_df, features_list, transform = None, target_transform = None):
        self.img_labels = merged_df
        self.features_list = features_list
        self.transform = transform
        self.target_transform = target_transform
        print("Dataset loaded. Length = ",len(self.features_list))


    def __len__(self):
        return len(self.features_list)

    def __getitem__(self, idx):
        feature = self.features_list[idx]
        
        label = self.img_labels.iloc[idx, 64:]
        if self.transform:
            feature = self.transform(feature)
        if self.target_transform:
            label = self.target_transform(label)
        return feature, label
    
    def get_all_data(self):
        return self.features_list

In [ ]:
from torchvision.transforms import ToTensor, ToPILImage


training_data = featuresDataset(
    merged_df=train_metadata_df,
    features_list=alex_features,
#    train=True,
#    download=True,
    #transform=ToTensor()
)


In [ ]:
training_data[55]